In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend import Legend 
from matplotlib import font_manager
from matplotlib import gridspec
import anndata as ad
import seaborn as sns
import scanpy as sc
import yaml
from pathlib import Path
import warnings
from collections import Counter
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, leaves_list

warnings.filterwarnings('ignore')

from scripts.atlas_plots import AtlasPlotting


In [ ]:
#adata = sc.read_h5ad("/mnt/storage/Daniele/atlases/human/Human_Atlas_Harmonised_Core.h5ad")
adata = ad.read_zarr("/mnt/ssd/atlases/Human_Atlas_Harmonised_Core.zarr")
adata.X = adata.layers['log_norm']

In [ ]:
adata.obs["Data_Source"] = np.where(adata.obs["Dataset"] == "in_house", "in house", "public data")

In [ ]:

#figgen = AtlasPlotting("config/config.yml",output_dir="/mnt/kkf2/Cell/AG-Saur/KKF2/Daniele/pdac_atlas_figures/figure2") pierre

figgen = AtlasPlotting("config/config.yml",output_dir="/mnt/t06/Cell/AG-Saur/KKF2/Daniele/pdac_atlas_figures/figure2")


In [ ]:
masked_plot = {
    "CD4+ T Cells": ["CD4+ Naive T Cell", "CD4+ Th1 Cell", "CD4+ Th2 Cell", "CD4+ Th17 Cell",
            "CD4+ Th22 Cell", "T-reg", "CD4+ Memory T Cell"],
    "CD8+ T Cells": ["CD8+ Naive T Cell", "CD8+ Effector T Cell", "CD8+ Terminal Effector T Cell",
            "CD8+ Exhausted T Cell", "CD8+ Tissue-Resident Memory T Cell", "CD8+ Memory T Cell"],
    "Mixed T Cells": ["Double Positive CD4+CD8+ T Cell", "γδ T Cell (Vδ1)"],
    "Malignant Cells": ["Malignant Cell - Pit Like", "Malignant Cell - Acinar-like", "Malignant Cell - Epithelial",
            "Malignant Cell - Hypoxia", "Malignant Cell - EMT", "Malignant Cell - Highly Proliferative",
            "Malignant Cell - Mesenchymal", "Malignant Cell - Highly Invasive", "Malignant Cell - Senescence",
            "Malignant Cell - Apoptotic"],
    "Macrophages": ["Macrophage - lipid processing TAM", "Macrophage - antigen processing TAM",
            "Macrophage - C1Q+ TAM", "Macrophage - angiogenic TAM", "Macrophage - CD3+ TAM"],
    "Neutrophils": ["Neutrophil - N0", "Neutrophil - N1", "Neutrophil - N2"],
            "Dendritic cells": ["Dendritic Cell - pDC", "Dendritic Cell - cDC1", "Dendritic Cell - cDC2"],

    "Malignant cells": ["Malignant Cell - Pit Like",
                        "Malignant Cell - Acinar-like",
                        "Malignant Cell - Epithelial",
                        "Malignant Cell - Hypoxia",
                        "Malignant Cell - EMT",
                        "Malignant Cell - Highly Proliferative",
                        "Malignant Cell - Mesenchymal",
                        "Malignant Cell - Highly Invasive",
                        "Malignant Cell - Senescence",
                        "Malignant Cell - Apoptotic"]
}

for figure_name, mask_values in masked_plot.items():
    figgen.create_masked_umap_highlight(adata, mask_column="Level_4", figure_name=f"{figure_name}_ordered", mask_values=mask_values)
    figgen.create_masked_umap_highlight(adata, mask_column="Level_4", figure_name=f"{figure_name}_ordered", mask_values=mask_values,highlight_size=1)
    figgen.create_masked_umap_highlight(adata, mask_column="Level_4", figure_name=f"{figure_name}_ordered", mask_values=mask_values,highlight_size=0.75)
    figgen.create_masked_umap_highlight(adata, mask_column="Level_4", figure_name=f"{figure_name}_ordered", mask_values=mask_values,highlight_size=0.5)
    figgen.create_masked_umap_highlight(adata, mask_column="Level_4", figure_name=figure_name, mask_values=mask_values,ordered=False)
    figgen.create_masked_umap(adata, mask_column="Level_4", figure_name=figure_name, mask_values=mask_values)


In [ ]:

figgen.create_masked_umap_highlight(adata, mask_column="TreatmentStatus", figure_name="TreatmentStatus", mask_values=["Treated"],color_by="TreatmentStatus")
figgen.create_masked_umap_highlight(adata, mask_column="TreatmentStatus", figure_name="TreatmentStatus", mask_values=["Treated"],color_by="TreatmentStatus",highlight_size=1)
figgen.create_masked_umap_highlight(adata, mask_column="TreatmentStatus", figure_name="TreatmentStatus", mask_values=["Treated"],color_by="TreatmentStatus",highlight_size=0.5)



In [ ]:
figgen.sample_and_cell_counts_barplot(
    adata,
    level_column="TreatmentType",
    sample_column="Sample_ID",
    custom_palette=True,
    save_name="sample_cell_counts_treatment",
    figsize=(10,10)
)


In [ ]:
figgen.sample_and_cell_counts_barplot_break_axis(
    adata,
    level_column="TreatmentType",
    sample_column="Sample_ID",
    custom_palette=True,
    save_name="cell_counts_treatment_broken_y",
    figsize=(7,5),
    break_point=200000,
    break_gap=0.1,
    plot_type="cells"
)

In [ ]:
figgen.sample_and_cell_counts_barplot_break_axis(
    adata,
    level_column="TreatmentType",
    sample_column="Sample_ID",
    custom_palette=True,
    save_name="sample_counts_treatment_broken_y",
    figsize=(7,5),
    break_point=30,
    break_gap=0.1,
    plot_type="samples"
)


In [ ]:
figgen.sample_and_cell_counts_barplot(
    adata,
    level_column="Treatment",
    sample_column="Sample_ID",
    save_name="sample_cell_counts_treatment",
    figsize=(10,10)
)


In [ ]:
figgen.sample_and_cell_counts_barplot(
    adata,
    level_column="TreatmentStatus",
    sample_column="Sample_ID",
    save_name="sample_cell_counts_treatment_status",
    figsize=(7,5),
    custom_palette=True,
    xlabel="Treatment status"
)

In [ ]:
fig = figgen.create_all_stacked_barplots(
    adata, 
    level_column='Level_4',
    subset_level='Level_2',
    subset_value="Malignant Cell",
    order_by_column='TreatmentStatus',
    save_name_prefix='malignant_cells_composition_Treatment_Status',
    xlabel=" ",
    figsize=(18, 6),
)

In [ ]:
fig = figgen.create_all_stacked_barplots(
    adata, 
    level_column='Level_4',
    subset_level='Level_2',
    subset_value="Malignant Cell",
    order_by_column='TreatmentType',
    save_name_prefix='malignant_cells_composition_Treatment_Type',
    xlabel=" ",
    figsize=(18, 6),
)

In [ ]:
fig = figgen.composition_heatmap(
        adata, 
    level_column='Level_4',
    subset_level='Level_2',
    subset_value="Malignant Cell",
    order_by_column='TreatmentStatus',
    title='Malignant cell composition',
    save_name='malignant_cells_composition_heatmap',
    figsize=(16,5)
)

In [ ]:
fig_counts, fig_rel = figgen.cell_abundance_barplot(
    adata,
    cell_type_column='Level_2',
    figsize=(10,5),
    save_name='cell_type_abundance_level2'
)

In [ ]:
fig_counts, fig_rel = figgen.cell_abundance_barplot(
    adata,
    cell_type_column='Level_3',
    figsize=(15,7),
    save_name='cell_type_abundance_level3'
)

In [ ]:
fig_counts, fig_rel = figgen.cell_abundance_barplot(
    adata,
    cell_type_column='Level_4',
    figsize=(20,10),
    save_name='cell_type_abundance_level4'
)

In [ ]:
sc.settings.figdir = "/mnt/t06/Cell/AG-Saur/KKF2/Daniele/pdac_atlas_figures/figure2/scanpy_umaps/"

In [ ]:
sc.pl.umap(adata,color="Level_1",frameon=False,palette=figgen.config["palettes"]["Level_1"],save="_level1_coloured.png")
sc.pl.umap(adata,color="Level_1",frameon=False,legend_loc=None,show=False,palette=figgen.config["palettes"]["Level_1"],save="_level1_coloured_nolegend.png")

In [ ]:
sc.pl.umap(adata,color="Level_2",frameon=False,palette=figgen.config["palettes"]["Level_2"],save="_level2_coloured.png")
sc.pl.umap(adata,color="Level_2",frameon=False,legend_loc=None,show=False,palette=figgen.config["palettes"]["Level_2"],save="_level2_coloured_nolegend.png")

In [ ]:
sc.pl.umap(adata,color="Level_3",frameon=False,palette=figgen.config["palettes"]["Level_3"],save="_level3_coloured.png")
sc.pl.umap(adata,color="Level_3",frameon=False,legend_loc=None,show=False,palette=figgen.config["palettes"]["Level_3"],save="_level3_coloured_nolegend.png")

In [ ]:
sc.pl.umap(adata,color="Treatment",frameon=False,save="_treatment_coloured.png")
sc.pl.umap(adata,color="Treatment",frameon=False,legend_loc=None,show=False,save="_Treatment_coloured_nolegend.png")

In [ ]:
sc.pl.umap(adata,color="Condition",frameon=False,save="_Condition_coloured.png")
sc.pl.umap(adata,color="Condition",frameon=False,legend_loc=None,show=False,save="_Condition_coloured_nolegend.png")

In [ ]:
sc.pl.umap(adata,color="TreatmentStatus",palette=figgen.config["palettes"]["TreatmentStatus"],title="Treatment Status",frameon=False,save="_TreatmentStatus_coloured.png")
sc.pl.umap(adata,color="TreatmentStatus",palette=figgen.config["palettes"]["TreatmentStatus"],title="Treatment Status",frameon=False,legend_loc=None,show=False,save="_Condition_coloured_nolegend.png")

In [ ]:
#figgen.matrix_markers(adata[adata.obs["Level_2"]=="Malignant Cell"],groupby_column="Level_4",save_name="malignant_markers")

In [ ]:
#figgen.matrix_markers(adata[adata.obs["Level_2"]=="Malignant Cell"],groupby_column="Level_4",n_markers=3,save_name="malignant_markers_n3")

In [ ]:
sc.settings.figdir = "/mnt/t06/Cell/AG-Saur/KKF2/Daniele/pdac_atlas_figures/figure2/marker_analysis/"

In [ ]:
#add json markers plot matrix
import json

with open('../../supplementary_data/human/annotation_markers.json', 'r') as f:
   data = json.load(f)



In [ ]:
figgen.matrix_annotation_markers(adata, data,"Malignant Cells", figsize=(12, 8),save_name="malignant_cells")


In [ ]:
figgen.matrix_annotation_markers(adata, data,"CD8 T cells", figsize=(12, 8),save_name="cd8_t_cells")
figgen.matrix_annotation_markers(adata, data,"CD4 T cells", figsize=(12, 8),save_name="cd4_t_cells")
figgen.matrix_annotation_markers(adata, data,"Other T cells", figsize=(12, 8),save_name="other_t_cells")
figgen.matrix_annotation_markers(adata, data,"Neutrophils", figsize=(12, 8),save_name="neutrophils")
figgen.matrix_annotation_markers(adata, data,"Macrophages and Monocytes", figsize=(12, 8),save_name="macrophages_monocytes")


In [ ]:
with open('../../supplementary_data/human/annotation_markers_updated.json', 'r') as f:
   two_marker_json = json.load(f)


In [ ]:
figgen.matrix_annotation_markers(adata, two_marker_json,"CD8 T cells", figsize=(12, 8),save_name="cd8_t_cells_top2")


In [ ]:
figgen.matrix_annotation_markers(adata, two_marker_json,"Malignant Cells", figsize=(12, 8),save_name="malignant_cells_top2")
figgen.matrix_annotation_markers(adata, two_marker_json,"CD8 T cells", figsize=(12, 8),save_name="cd8_t_cells_top2")
figgen.matrix_annotation_markers(adata, two_marker_json,"CD4 T cells", figsize=(12, 8),save_name="cd4_t_cells_top2")
figgen.matrix_annotation_markers(adata, two_marker_json,"Other T cells", figsize=(12, 8),save_name="other_t_cells_top2")
figgen.matrix_annotation_markers(adata, two_marker_json,"Neutrophils", figsize=(12, 8),save_name="neutrophils_top2")
figgen.matrix_annotation_markers(adata, two_marker_json,"Macrophages and Monocytes", figsize=(12, 8),save_name="macrophages_monocytes_top2")

In [ ]:
figgen.stacked_violin_annotation_markers(adata, two_marker_json,"Malignant Cells", figsize=(12, 8),save_name="malignant_cells_top2")
figgen.stacked_violin_annotation_markers(adata, two_marker_json,"CD8 T cells", figsize=(12, 8),save_name="cd8_t_cells_top2")
figgen.stacked_violin_annotation_markers(adata, two_marker_json,"CD4 T cells", figsize=(12, 8),save_name="cd4_t_cells_top2")
figgen.stacked_violin_annotation_markers(adata, two_marker_json,"Other T cells", figsize=(12, 8),save_name="other_t_cells_top2")
figgen.stacked_violin_annotation_markers(adata, two_marker_json,"Neutrophils", figsize=(12, 8),save_name="neutrophils_top2")
figgen.stacked_violin_annotation_markers(adata, two_marker_json,"Macrophages and Monocytes", figsize=(12, 8),save_name="macrophages_monocytes_top2")

In [ ]:
markers_flat = ["CD3E",'C1QA',
 'CD163',
 'CD74',
 'IL1B',
 'VCAN',
 'SLC2A3',
 'APOE',
 'TREM2',
 'DPYD',
 'ELMO1']

In [ ]:

celltypes_list = sorted(two_marker_json["Macrophages and Monocytes"]["markers"].keys())
fig = plt.figure(figsize=(12, 8), dpi=300)


sc.pl.stacked_violin(adata[adata.obs["Level_4"].isin(celltypes_list)],
            standard_scale="var",layer="log_norm",var_names=markers_flat,
            groupby="Level_4",cmap="coolwarm",ax=fig.gca())
plt.tight_layout()

fig.savefig("/mnt/t06/Cell/AG-Saur/KKF2/Daniele/pdac_atlas_figures/figure2/marker_analysis/macrophages_monocytes_top2_stacked_violin_json_markers.png",
            dpi=300,
            bbox_inches='tight', 
            facecolor='white')


In [ ]:
fig = plt.figure(figsize=(12, 8), dpi=300)


sc.pl.matrixplot(adata[adata.obs["Level_4"].isin(celltypes_list)],
            standard_scale="var",layer="log_norm",var_names=markers_flat,
            groupby="Level_4",cmap="coolwarm",ax=fig.gca())
plt.tight_layout()

fig.savefig("/mnt/t06/Cell/AG-Saur/KKF2/Daniele/pdac_atlas_figures/figure2/marker_analysis/macrophages_monocytes_top2_matrixplot_json_markers.png",
            dpi=300,
            bbox_inches='tight', 
            facecolor='white')
